由于正负样本不均衡，故提出以下方案

**解决方案**
1. 采用分治集成模型。将负类数据通过高斯混合聚类算法划分为15个块，使用负类中的一个块与正类数据训练模型，可获得15个模型。取预测的概率值的均值，作为最终的判断

2. 以auc_roc指标作为优化目标训练模型

3. 训练一个神经网络模型

读取数据

In [3]:
import gc
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

matrix = pd.read_csv('./result/matrix.csv')
matrix = matrix.drop(matrix.columns[0],axis=1)

#train、test-setdata
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']

#导入分析库
from sklearn.model_selection import train_test_split
# X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.3)
# 最终版提交，使用所有训练数据，训练模型
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.1)
X_train, X_valid, y_train, y_valid = X_train.values, X_valid.values, y_train.values.astype('float32'), y_valid.values.astype('float32') 
test_data = test_data.values

gc.collect()
# del data_train,temp

18

### 方案一: 聚合模型

In [ ]:
import time
import xgboost as xgb
from model import model_train
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler


model_dict = {'svm':SVC,'logistic':LogisticRegression,'decision_tree':DecisionTreeClassifier,'bayes':GaussianNB,
               'mixture':GaussianMixture,'KNN':KNeighborsClassifier,'GBDT':GradientBoostingClassifier,'XGBoost':XGBClassifier}

std = StandardScaler()
std = std.fit(X_train)
clf = model_dict['mixture'](n_components=15)
clf = clf.fit(std.transform(X_train))
x_train_cat = clf.predict(X_train)

import matplotlib.pyplot as plt
import seaborn as sns
sns.countplot(x_train_cat)
plt.show()

# 结果不理想
indices_11 = (x_train_cat==11)
X_train_11 = X_train[indices_11]
y_train_11 = y_train[indices_11]
model_train('logistic',model=model_dict['logistic'],x_train=X_train_11,y_train=y_train_11,x_val=X_valid,y_val=y_valid)
model_train('bayes',model=model_dict['bayes'],x_train=X_train_11,y_train=y_train_11,x_val=X_valid,y_val=y_valid)
model_train('decision_tree',model=model_dict['decision_tree'],x_train=X_train_11,y_train=y_train_11,x_val=X_valid,y_val=y_valid)
model_train('XGBoost',model=model_dict['XGBoost'],x_train=X_train_11,y_train=y_train_11,x_val=X_valid,y_val=y_valid)

del X_train_11,y_train_11,x_train_cat

方案一更正：
使用高斯混合大多数的类别都聚集在第12个类别上，样本本质上还是不平衡。故采取随机选择的策略获取15个块

In [6]:
false_indices = (y_train==0)
true_indices = (y_train==1)

x_train_true = X_train[true_indices]
y_train_true = y_train[true_indices]

x_train_false = X_train[false_indices]
y_train_false = y_train[false_indices]

true_count = y_train_true.shape[0]
x_train_false_list = []
y_train_false_list = []
for i in range(15):
    if i < 14:
        x_train_false_list.append(x_train_false[true_count*i:(i+1)*true_count])
        y_train_false_list.append(y_train_false[true_count*i:(i+1)*true_count])
    else:
        x_train_false_list.append(x_train_false[true_count*i:])
        y_train_false_list.append(y_train_false[true_count*i:])

x_train_list = []
y_train_list = []
for i in range(len(x_train_false_list)):
    x_train_i = x_train_false_list[i]
    y_train_i = y_train_false_list[i]
    # 将正负类进行连接，并打乱
    x_train_son = np.concatenate((x_train_i,x_train_true),axis=0)
    y_train_son = np.concatenate((y_train_i,y_train_true),axis=0)
    shuffler = np.random.permutation(len(x_train_son))
    x_train_son = x_train_son[shuffler]
    y_train_son = y_train_son[shuffler]
    x_train_list.append(x_train_son)
    y_train_list.append(y_train_son)

del x_train_true,x_train_false,y_train_true,y_train_false,x_train_false_list,y_train_false_list
gc.collect()


2170

**集成模型**

1. 聚合模型：对这15个模型的预测概率值，做均值处理

2. 将聚合模型用于验证集的测试

3. 生成submission,观察效果。 分数：使用GBDT,score:0.6654492; 使用XGB,score:0.6701968

In [8]:
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

model_agg = []
roc_list = []

def model_train_i(clf,x_train_i,y_train_i,X_valid,y_valid):
    clf = clf.fit(x_train_i,y_train_i)
    y_val_pred = clf.predict_proba(X_valid)
    roc = roc_auc_score(y_true=y_valid, y_score=y_val_pred[:,1])    
    return clf,roc

for i in range(len(x_train_list)):
    x_train_i = x_train_list[i]
    y_train_i = y_train_list[i]
    XGB = model_dict['XGBoost'](
    # max_depth=8,
    max_depth=6,
    n_estimators=300,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42,
    n_jobs=-1)
    XGB, roc_X = model_train_i(XGB,x_train_i,y_train_i,X_valid,y_valid)
    model_agg.append({'XGB':XGB})
    print (f'第{i+1}个模型选择了XGB,其AUC值为{roc_X}')
    # if roc_G > roc_X:
    #     model_agg.append({'GBDT':GBDT})
    #     print (f'第{i+1}个模型选择了GBDT,其AUC值为{roc_G}')
    # else:
    #     model_agg.append({'XGB':XGB})
    #     print (f'第{i+1}个模型选择了XGB,其AUC值为{roc_X}')

[18:00:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
第1个模型选择了XGB,其AUC值为0.6959346769569423
[18:00:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
第2个模型选择了XGB,其AUC值为0.6957330795639762
[18:00:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavi

In [9]:
from sklearn.metrics import accuracy_score ,roc_auc_score

model_agg = []
roc_list = []

def model_train_i(clf,x_train_i,y_train_i,X_valid,y_valid):
    clf = clf.fit(x_train_i,y_train_i)
    y_val_pred = clf.predict_proba(X_valid)
    roc = roc_auc_score(y_true=y_valid, y_score=y_val_pred[:,1])    
    return clf,roc

for i in range(len(x_train_list)):
    x_train_i = x_train_list[i]
    y_train_i = y_train_list[i]
    XGB = model_dict['XGBoost'](
    # max_depth=8,
    max_depth=8,
    n_estimators=100,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42,
    n_jobs=-1)
    XGB, roc_X = model_train_i(XGB,x_train_i,y_train_i,X_valid,y_valid)
    model_agg.append({'XGB':XGB})
    print (f'第{i+1}个模型选择了XGB,其AUC值为{roc_X}')
    # if roc_G > roc_X:
    #     model_agg.append({'GBDT':GBDT})
    #     print (f'第{i+1}个模型选择了GBDT,其AUC值为{roc_G}')
    # else:
    #     model_agg.append({'XGB':XGB})
    #     print (f'第{i+1}个模型选择了XGB,其AUC值为{roc_X}')

[18:02:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
第1个模型选择了XGB,其AUC值为0.7059993912202367
[18:02:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
第2个模型选择了XGB,其AUC值为0.7107033303894447
[18:02:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavi

**聚合模型总结**
1. 由于样本不平衡，故使用此聚合模型可提升AUC值

2. 试验后发现，该聚合模型能在baseline的基础上提升2-4个百分点。后期的迭代优化中，应考虑与其他方案进行融合

In [10]:
def agg_model(model_agg,X_valid):
    val_pred = None
    for i in range(len(model_agg)):
        clf = list(model_agg[i].values())[0]
        y_valid_pred = np.expand_dims(clf.predict_proba(X_valid)[:,1],axis=0)
        if i == 0:
            val_pred = y_valid_pred
        else:
            val_pred = np.concatenate((val_pred,y_valid_pred),axis=0)
    val_pred = val_pred.mean(axis=0)
    return val_pred

val_pred = agg_model(model_agg,X_valid)
print (f'验证集上表现出AUC:{roc_auc_score(y_valid,val_pred)}')

验证集上表现出AUC:0.7168796626529258


生成提交文件

In [11]:
paths = './data_format1'
submission = pd.read_csv(f'{paths}/test_format1.csv')

prob = agg_model(model_agg,test_data)
submission['prob'] = prob
# submission.dorp(['origin'], axis=1, inplace=True)
submission.to_csv('./result/submission_agg.csv', index=False)

### 方案二：神经网络模型

数据转化为tensor,并归一化

In [ ]:
import torch
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std = std.fit(X_train)
X_train, X_valid = std.transform(X_train), std.transform(X_valid)
test_data = std.transform(test_data)
# 将原始数据转化为tensor类型
device = torch.device('cuda')
X_train_t = torch.Tensor(X_train).to(device)
y_train_t = torch.Tensor(y_train).long().to(device)
X_valid_t = torch.Tensor(X_valid).to(device)
y_valid_t = torch.Tensor(y_valid).long().to(device)
X_test_t = torch.Tensor(test_data).to(device)

神经网络模型构建

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_auc_score

device = torch.device('cuda')
class Net(nn.Module):
    def __init__(self,in_dim,hidden_dim_list,out_dim=2):
        super(Net,self).__init__()
        self.fc_list = []
        for i in range(len(hidden_dim_list)):
            hidden_dim = hidden_dim_list[i]
            if i == 0:
                self.fc_list.append(nn.Linear(in_dim,hidden_dim))
                self.fc_list.append(nn.BatchNorm1d(hidden_dim))
                self.fc_list.append(nn.Dropout())
                self.fc_list.append(nn.ReLU())
            else:
                self.fc_list.append(nn.Linear(hidden_dim_list[i-1],hidden_dim))
                self.fc_list.append(nn.BatchNorm1d(hidden_dim))
                self.fc_list.append(nn.Dropout())
                self.fc_list.append(nn.ReLU())
        self.fc_list.append(nn.Linear(hidden_dim_list[-1],out_dim))
        self.layers = nn.Sequential(*self.fc_list)

    def forward(self,x):    
        out = self.layers(x)
        out = torch.sigmoid(out)
        return out

in_dim = X_train_t.shape[1]
hidden_dim_list = [68]
out_dim = 2

net = Net(in_dim,hidden_dim_list,out_dim).to(device)
optimizer = optim.Adam(net.parameters(),lr=0.001)
loss_func = nn.CrossEntropyLoss()

模型训练

In [ ]:
def net_train(net,X_train,y_train,batch_size=32,epoch=1,verbose=1):
    net.train()
    batch = int(X_train.shape[0]/batch_size)
    for i in range(epoch):
        for j in range(batch):
            optimizer.zero_grad()
            X_train_1 = X_train[j*batch_size:(j+1)*batch_size,:]
            y_train_1 = y_train[j*batch_size:(j+1)*batch_size]
            pred = net(X_train_1)
            loss = loss_func(pred,y_train_1)
            loss.backward()
            optimizer.step()
            if j % 1000 == 0 and verbose == 1:
                print (f'第{j}个batch, 损失值为{loss}')
        if verbose == 1:
            print (f'\nepoch {i+1}:loss equals to {loss}')
# net_train(net,X_train_t,y_train_t,batch_size=120,epoch=5)

神经网络在验证的时候，全部的类都显示为父类，模型没有效果。

**原因**：
1. 神经网络对正负样本的比例要求严格。该数据集验证不平衡，故训练效果非常差

**解决**
1. 将方案一的方法与神经网络进行融合

In [ ]:
# 估计网络没有学习到
def net_eval(net,X_valid,y_valid):
    net.eval()
    pred = net(X_valid)
    # torch.argmax(pred)[0]
    pred = pred[:,0].cpu().detach().numpy()
    print ((pred==0).sum())
    print ((pred==1).sum())
    print (f'验证集上, roc的值为{roc_auc_score(y_valid.cpu().detach().numpy(),pred)}')

net_eval(net,X_valid_t,y_valid_t)

按照方案一的方式，将数据集的正负样本均衡化

防止正负样本比例失调，对模型造成影响

In [ ]:
false_indices = (y_train==0)
true_indices = (y_train==1)

x_train_true = X_train[true_indices]
y_train_true = y_train[true_indices]

x_train_false = X_train[false_indices]
y_train_false = y_train[false_indices]

true_count = y_train_true.shape[0]
x_train_false_list = []
y_train_false_list = []
for i in range(15):
    if i < 14:
        x_train_false_list.append(x_train_false[true_count*i:(i+1)*true_count])
        y_train_false_list.append(y_train_false[true_count*i:(i+1)*true_count])
    else:
        x_train_false_list.append(x_train_false[true_count*i:])
        y_train_false_list.append(y_train_false[true_count*i:])

x_train_list = []
y_train_list = []
for i in range(len(x_train_false_list)):
    x_train_i = x_train_false_list[i]
    y_train_i = y_train_false_list[i]
    # 将正负类进行连接，并打乱
    x_train_son = np.concatenate((x_train_i,x_train_true),axis=0)
    y_train_son = np.concatenate((y_train_i,y_train_true),axis=0)
    shuffler = np.random.permutation(len(x_train_son))
    x_train_son = x_train_son[shuffler]
    y_train_son = y_train_son[shuffler]
    x_train_list.append(x_train_son)
    y_train_list.append(y_train_son)

del x_train_true,x_train_false,y_train_true,y_train_false,x_train_false_list,y_train_false_list
gc.collect()

即便使用这20000个样本来训练，样本虽然均衡了，但是预测效果不怎么样，其对应的auc的值也很低

综合分析，放弃方案二

In [ ]:
import torch
from sklearn.preprocessing import StandardScaler

X_valid_t = torch.Tensor(X_valid).to(device)
y_valid_t = torch.Tensor(y_valid).long().to(device)
X_test_t = torch.Tensor(test_data).to(device)
net = Net(in_dim,hidden_dim_list,out_dim).to(device)

# 将原始数据转化为tensor类型
for i in range(len(x_train_list)):
    if i == 0:
        device = torch.device('cuda')
        X_train_i = x_train_list[i]
        y_train_i = y_train_list[i]

        X_train_t = torch.Tensor(x_train_i).to(device)
        y_train_t = torch.Tensor(y_train_i).long().to(device)
        net_train(net,X_train_t,y_train_t,batch_size=2,epoch=1,verbose=0)
        net_eval(net,X_valid_t,y_valid_t)
        break

### 方案三：过采样，使正负样本均衡后，再预测算法

1. 使用smote进行过采样

In [12]:
from model import model_train
from imblearn.over_sampling import SMOTE
# 过采样这么快，震惊我一整年

smote = SMOTE(sampling_strategy='minority')
X_smo,y_smo = smote.fit_sample(X_train,y_train)
X_val_smo,y_val_smo = smote.fit_sample(X_valid,y_valid)

Using TensorFlow backend.


In [13]:
# model_train('logistic',model=model_dict['logistic'],x_train=X_smo,y_train=y_smo,x_val=X_valid,y_val=y_valid)
# model_train('bayes',model=model_dict['bayes'],x_train=X_smo,y_train=y_smo,x_val=X_valid,y_val=y_valid)
# model_train('decision_tree',model=model_dict['decision_tree'],x_train=X_smo,y_train=y_smo,x_val=X_valid,y_val=y_valid)
# model_train('GBDT',model=model_dict['GBDT'],x_train=X_train,y_train=y_train,x_val=X_valid,y_val=y_valid)
model_train('XGBoost',model=model_dict['XGBoost'],x_train=X_smo,y_train=y_smo,x_val=X_smo,y_val=y_smo)
model_train('XGBoost',model=model_dict['XGBoost'],x_train=X_smo,y_train=y_smo,x_val=X_val_smo,y_val=y_val_smo)
model_train('XGBoost',model=model_dict['XGBoost'],x_train=X_smo,y_train=y_smo,x_val=X_valid,y_val=y_valid)

[18:42:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
模型XGBoost:
模型XGBoost, 耗时 1.895048983891805 minutes:
使用clf.score计算, 测试集的准确度为0.9663319979856363
测试集的ROC面积为0.9838324282423343
测试集的混淆矩阵为:
[[220281    136]
 [ 14706 205711]]
测试集的分类报告为:
              precision    recall  f1-score   support

         0.0       0.94      1.00      0.97    220417
         1.0       1.00      0.93      0.97    220417

    accuracy                           0.97    440834
   macro avg       0.97      0.97      0.97    440834
weighted avg       0.97      0.97      0.97    440834

[18:44:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8,
              enable_categorical=False, eta=0.3, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=300, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              subsample=0.8, tree_method='exact', use_label_encoder=True, ...)

2. 重复过采样

In [14]:
# X_smo,y_smo = smote.fit_sample(X_train,y_train)

X_train_true_rep = np.repeat(X_train[(y_train==1)],15,axis=0)
x_train_false_rep = X_train[(y_train==0)]
y_train_true_rep = np.ones(X_train_true_rep.shape[0])
y_train_false_rep = np.zeros(x_train_false_rep.shape[0])


x_train_rep = np.concatenate((X_train_true_rep,x_train_false_rep),axis=0)
y_train_rep = np.concatenate((y_train_true_rep,y_train_false_rep),axis=0)

indices = np.random.permutation(x_train_rep.shape[0])
x_train_rep = x_train_rep[indices]
y_train_rep = y_train_rep[indices]
del X_train_true_rep,x_train_false_rep,y_train_true_rep,y_train_false_rep

print (y_train_rep.shape)
print ((y_train_rep==0).sum(),(y_train_rep==1).sum())

(435817,)
220417 215400


In [15]:
model = model_train('XGBoost',model=model_dict['XGBoost'],x_train=x_train_rep,y_train=y_train_rep,x_val=x_train_rep,y_val=y_train_rep)
model = model_train('XGBoost',model=model_dict['XGBoost'],x_train=x_train_rep,y_train=y_train_rep,x_val=X_valid,y_val=y_valid)
# model_train('XGBoost',model=model_dict['XGBoost'],x_train=x_train_rep,y_train=y_train_rep,x_val=X_valid,y_val=y_valid)

[18:47:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
模型XGBoost:
模型XGBoost, 耗时 0.9007354537645976 minutes:
使用clf.score计算, 测试集的准确度为0.7479515484710325
测试集的ROC面积为0.8281732020717936
测试集的混淆矩阵为:
[[162170  58247]
 [ 51600 163800]]
测试集的分类报告为:
              precision    recall  f1-score   support

         0.0       0.76      0.74      0.75    220417
         1.0       0.74      0.76      0.75    215400

    accuracy                           0.75    435817
   macro avg       0.75      0.75      0.75    435817
weighted avg       0.75      0.75      0.75    435817

[18:48:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

In [ ]:
paths = './data_format1'
submission = pd.read_csv(f'{paths}/test_format1.csv')

prob = model(test_data)[:,1]
submission['prob'] = prob
# submission.dorp(['origin'], axis=1, inplace=True)
submission.to_csv('./result/submission_oversample.csv', index=False)

方案三总结：

1. 过采样在训练集能获得很高的准确度与auc面积，但是这是基于正负样本的属于1：1的比例

2. 验证集上测试，表现效果异常的差。

3. 该方案也不能有效提升auc指标的值。同样抛弃


### 项目总结

1. 最终发现能用的也只有方案一，提交给天池平台最高能达到0.6871的auc值。

2. 方案二、三体现的是思考过程，在此项目中就是关于不平衡数据处理的思考。
